In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [10]:
df = spark.read.format("csv").option("header","true").load("cars.csv")

In [11]:
df.show(5)

+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission| color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+----

In [12]:
df.show(1,vertical=True)

-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

In [ ]:
# смотрим на DataFrame API
# выбираем несколько колонок для отображения

In [20]:
df.select("manufacturer_name","model_name").show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



In [13]:
import pyspark.sql.functions as F
df.select(F.col("manufacturer_name"),F.col("model_name")).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



In [ ]:
# filter() it where условие для фильтрации ао audi

In [34]:
df.select("manufacturer_name","model_name","transmission").filter("manufacturer_name = 'Audi'").show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|        Q7|   automatic|
|             Audi|        TT|   automatic|
|             Audi|       100|  mechanical|
|             Audi|        A6|   automatic|
|             Audi|        Q3|   automatic|
+-----------------+----------+------------+
only showing top 5 rows



In [35]:
# выбираем несколько из нескольки условий.просто через точку записывать.filter
df.select("manufacturer_name","model_name","transmission").filter("manufacturer_name = 'Audi'").filter("transmission = 'mechanical'").show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



In [ ]:
# удобней и логичнее использовать col() для составления условий фильтрации:
name_avto = 'audio'
df.select("manufacturer_name","model_name","transmission")\
.filter(F.col("manufacturer_name") == name_avto)\
.filter(F.col("transmission") == 'mechanical')\
.show(5)

In [ ]:
df.count()

In [38]:
df.select("manufacturer_name").distinct().count()

55

In [40]:
# группировка
# мы возвращаем DataFrame и по этому надо навесить .show() чтобы что-то посмотреть
df.groupBy("manufacturer_name").count().show() 

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|            Lexus|  213|
|           Jaguar|   53|
|            Rover|  235|
|           Lancia|   92|
|             Jeep|  107|
|       Mitsubishi|  887|
|              Kia|  912|
|             Mini|   68|
|            Lifan|   47|
|             LADA|  146|
|        SsangYong|   79|
|             Audi| 2468|
|             Seat|  303|
|         Cadillac|   43|
|          Москвич|   55|
|       Alfa Romeo|  207|
|            Geely|   71|
|          Renault| 2493|
|           Daewoo|  221|
+-----------------+-----+
only showing top 20 rows



In [44]:
# сортировка по убыванию
df.groupBy("manufacturer_name").count().orderBy("count").show(5) 

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Great Wall|   36|
|          Lincoln|   36|
|          Pontiac|   42|
|              ЗАЗ|   42|
|         Cadillac|   43|
+-----------------+-----+
only showing top 5 rows



In [46]:
df.groupBy("manufacturer_name").count().orderBy(F.col("count").desc()).show(5) 

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows



In [50]:
# ПЕРЕМЕНОВАТЬ. Первый аргумент как называлась колонка, второй аргумент как назвали по другому
df.withColumnRenamed("manufacturer_name","manufacturer").select('manufacturer').show(5)

+------------+
|manufacturer|
+------------+
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
+------------+
only showing top 5 rows



In [51]:
df.withColumn("next_year",F.col("year_produced")).select('year_produced').show(5)

+-------------+
|year_produced|
+-------------+
|         2010|
|         2002|
|         2001|
|         1999|
|         2001|
+-------------+
only showing top 5 rows



In [53]:
# вывести схему датафрейма (типы колонок)
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)


In [55]:
# вывести сводную статистику по датафрейму
df.select('manufacturer_name','year_produced').describe().show()

+-------+-----------------+------------------+
|summary|manufacturer_name|     year_produced|
+-------+-----------------+------------------+
|  count|            38531|             38531|
|   mean|             NULL|2002.9437336170874|
| stddev|             NULL| 8.065730511309367|
|    min|            Acura|              1942|
|    max|              УАЗ|              2019|
+-------+-----------------+------------------+



In [ ]:
# остановка сессии
spark.stop()